In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

#!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
#!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.0.0+cpu.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install warnings
!pip install dgl
!pip install texttable

if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(device)

2.0.1+cu118
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 627.0/627.0 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.1/504.1 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 732.3/732.3 kB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.7/205.7 kB 19.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement warnings (from versions: none)
ERROR: No matching distribution found for warnings
Looking in indexes: https://pypi.org/simple

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.utils.data
import scipy
import pickle
from skimage.segmentation import slic
from torchvision import datasets
import multiprocessing as mp
from torch_geometric.utils import from_networkx,to_networkx
from torch.utils.data import random_split
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GraphConv, BatchNorm, global_mean_pool
from torch_geometric.utils import softmax, degree

#/content/drive/MyDrive/Colab_Notebooks
import os
import random
import numpy as np
import os.path as osp
from torch.autograd import grad
from drive.MyDrive.Colab_Notebooks.DIR_GNN.utils.mask import set_masks, clear_masks
from drive.MyDrive.Colab_Notebooks.DIR_GNN.utils.logger import Logger
from datetime import datetime
from drive.MyDrive.Colab_Notebooks.DIR_GNN.utils.helper import random_partition, set_seed
from drive.MyDrive.Colab_Notebooks.DIR_GNN.utils.get_subgraph import split_graph, relabel
import scipy.ndimage
import scipy.spatial
import argparse
import datetime
import networkx as nx
import dgl
import os
import pickle
from scipy.spatial.distance import cdist
import numpy as np
import itertools

import csv
from sklearn.model_selection import StratifiedShuffleSplit
import random
from torchvision import transforms, datasets

import time

from scipy import ndimage



import torch


import matplotlib.pyplot as plt
import matplotlib

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [3]:
import argparse
import os.path as osp
import time
import sys
import torch
from torch.nn import Linear, ReLU, CrossEntropyLoss
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.nn import ARMAConv, global_mean_pool
from collections import OrderedDict
import torch.nn.functional as F
from torch.nn import ModuleList

class GraphSST2Net(torch.nn.Module):
    def __init__(self, in_channels, hid_channels=128, num_classes=2, num_layers=1):
        super(GraphSST2Net, self).__init__()

        self.convs = ModuleList([
            ARMAConv(in_channels, hid_channels),
            ARMAConv(hid_channels, hid_channels, num_layers=num_layers)])
        self.causal_mlp = torch.nn.Sequential(
            Linear(hid_channels, 2*hid_channels),
            ReLU(),
            Linear(2*hid_channels, num_classes)
        )
        self.conf_mlp = torch.nn.Sequential(
            Linear(hid_channels, 2*hid_channels),
            ReLU(),
            Linear(2*hid_channels, num_classes)
        )

    def forward(self, x, edge_index, edge_attr, batch):
        node_x = self.get_node_reps(x, edge_index, edge_attr, batch)
        graph_x = global_mean_pool(node_x, batch)
        return self.get_causal_pred(graph_x)

    def get_node_reps(self, x, edge_index, edge_attr, batch):
        edge_weight = edge_attr.view(-1)
        x = F.relu(self.convs[0](x, edge_index, edge_weight))
        node_x = self.convs[1](x, edge_index, edge_weight)
        return node_x

    def get_graph_rep(self, x, edge_index, edge_attr, batch):
        node_x = self.get_node_reps(x, edge_index, edge_attr, batch)
        graph_x = global_mean_pool(node_x, batch)
        return graph_x

    def get_causal_pred(self, graph_x):
        return self.causal_mlp(graph_x)

    def get_conf_pred(self, graph_x):
        return self.conf_mlp(graph_x)

    def get_comb_pred(self, causal_graph_x, conf_graph_x):
        causal_pred = self.causal_mlp(causal_graph_x)
        conf_pred = self.conf_mlp(conf_graph_x).detach()
        return torch.sigmoid(conf_pred) * causal_pred

    def reset_parameters(self):
        with torch.no_grad():
            for param in self.parameters():
                param.uniform_(-1.0, 1.0)


In [4]:
class CausalAttNet(nn.Module):

    def __init__(self, causal_ratio):
        super(CausalAttNet, self).__init__()
        channels=128
        self.conv1 = ARMAConv(in_channels=768, out_channels=channels)
        self.conv2 = ARMAConv(in_channels=channels, out_channels=channels)
        self.mlp = nn.Sequential(
            nn.Linear(channels*2, channels*4),
            nn.ReLU(),
            nn.Linear(channels*4, 1)
        )
        self.ratio = causal_ratio

    def forward(self, data):
        x = F.relu(self.conv1(data.x, data.edge_index, data.edge_attr.view(-1)))
        x = self.conv2(x, data.edge_index, data.edge_attr.view(-1))

        row, col = data.edge_index
        edge_rep = torch.cat([x[row], x[col]], dim=-1)
        edge_score = self.mlp(edge_rep).view(-1)

        (causal_edge_index, causal_edge_attr, causal_edge_weight), \
        (conf_edge_index, conf_edge_attr, conf_edge_weight) = split_graph(data,edge_score, self.ratio)

        causal_x, causal_edge_index, causal_batch, _ = relabel(x, causal_edge_index, data.batch)
        conf_x, conf_edge_index, conf_batch, _ = relabel(x, conf_edge_index, data.batch)

        return (causal_x, causal_edge_index, causal_edge_attr, causal_edge_weight, causal_batch),\
                (conf_x, conf_edge_index, conf_edge_attr, conf_edge_weight, conf_batch),\
                edge_score

In [ ]:
#len(new_012)

In [5]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from drive.MyDrive.gsst2.graphsst2_dataset import get_dataset, get_dataloader
path ='/content/drive/MyDrive/gsst2/'
dataset = get_dataset(dataset_dir=path, dataset_name='Graph_SST2', task=None)
dataloader = get_dataloader(dataset,
                            batch_size=128,
                            degree_bias=True)

#Epoch=2
#alpha_prime = 1e-4 * (Epoch ** 1.6)
causal_ratio=0.8
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
g = GraphSST2Net(128).to(device)
att_net = CausalAttNet(causal_ratio).to(device)
model_optimizer = torch.optim.Adam(
            list(g.parameters()) +
            list(att_net.parameters()),
            lr=0.001)
conf_opt = torch.optim.Adam(g.conf_mlp.parameters(), lr=0.001)
CELoss = nn.CrossEntropyLoss(reduction="mean")
train_loader = dataloader['train']
val_loader = dataloader['eval']
test_loader = dataloader['test']
EXp=500


28327 3147 12305


/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [6]:
import time
import json

loss_value=[]
loss_value_valation=[]
from tqdm import tqdm
for epoch in range(EXp):
  g.train()
  att_net.train()
  all_loss, n_bw, all_env_loss = 0, 0, 0
  all_causal_loss, all_conf_loss = 0, 0
  causal_edge_weights = torch.tensor([]).to(device)
  conf_edge_weights = torch.tensor([]).to(device)
  alpha_prime = 1e-4 * (epoch ** 1.6)
  dummy_w = nn.Parameter(torch.Tensor([1.0])).to(device)
  print(f"-----training-------{epoch}")
  loop = tqdm(enumerate(train_loader),total=len(train_loader))
  for it, graph in loop:
      graph.to(device)
      n_bw += 1
      N = graph.num_graphs
      (causal_x, causal_edge_index, causal_edge_attr, causal_edge_weight, causal_batch),\
      (conf_x, conf_edge_index, conf_edge_attr, conf_edge_weight, conf_batch), edge_score = att_net(graph)

      set_masks(causal_edge_weight, g)
      causal_rep = g.get_graph_rep(x=causal_x, edge_index=causal_edge_index,edge_attr=causal_edge_attr, batch=causal_batch)
      causal_out = g.get_causal_pred(causal_rep)
      #print(causal_out)
      clear_masks(g)
      set_masks(conf_edge_weight, g)
      conf_rep = g.get_graph_rep(
          x=conf_x, edge_index=conf_edge_index,
          edge_attr=conf_edge_attr, batch=conf_batch).detach()
      conf_out = g.get_conf_pred(conf_rep)
      clear_masks(g)
      #y=to01(graph.y).to(device)
      #print(conf_out)
      causal_loss = CELoss(causal_out,graph.y)
      #print(causal_loss)

      conf_loss = CELoss(conf_out, graph.y)
      #print('noncausal',float(conf_loss))
      #all_conf_loss += conf_loss
      #all_causal_loss += causal_loss
      #all_env_loss += env_loss
      #print(all_causal_loss)
      env_loss = 0


      env_loss = torch.tensor([]).to(device)
      for conf in conf_rep:
          rep_out = g.get_comb_pred(causal_rep, conf)
          env_loss = torch.cat([env_loss, CELoss(rep_out, graph.y).unsqueeze(0)])
      causal_loss += min(alpha_prime, 1) * env_loss.mean()
      env_loss = alpha_prime * torch.var(env_loss * conf_rep.size(0))

      all_conf_loss += conf_loss
      all_causal_loss += causal_loss
      all_env_loss += env_loss
      causal_edge_weights = torch.cat([causal_edge_weights, causal_edge_weight])
      conf_edge_weights = torch.cat([conf_edge_weights, conf_edge_weight])

  all_env_loss /= n_bw
  all_causal_loss /= n_bw
  all_conf_loss /= n_bw
  all_loss = all_causal_loss + all_env_loss

  loss_value.append(all_loss.item())
  print(f'number of {epoch} of all loss:{all_loss}')
  conf_opt.zero_grad()
  all_conf_loss.backward()
  conf_opt.step()

  model_optimizer.zero_grad()
  all_loss.backward()
  model_optimizer.step()

  with torch.no_grad():
    g.eval()
    att_net.eval()
    acc = 0
    for graph in test_loader:
        graph.to(device)
        (causal_x, causal_edge_index, causal_edge_attr, causal_edge_weight, causal_batch),\
        (conf_x, conf_edge_index, conf_edge_attr, conf_edge_weight, conf_batch), edge_score = att_net(graph)
        set_masks(causal_edge_weight, g)
        out = g(x=causal_x, edge_index=causal_edge_index,
                edge_attr=causal_edge_attr, batch=causal_batch)
        clear_masks(g)
        acc += torch.sum(out.argmax(-1).view(-1) == graph.y.view(-1))
    acc = float(acc) / len(test_loader.dataset)
    loss_value_valation.append(acc)
    print(f"causal val accuracy:{acc}")
    dictionary={"number of epoch":epoch,
                "training loss list":loss_value,
                "valation accuracy list":loss_value_valation}

    # Serializing json
    json_object = json.dumps(dictionary,indent=3)

    # Writing to sample.json
    with open("/content/drive/MyDrive/running_RQ2/running_Graph_sst2/running_DIR/Gsst2DIR_tl_va_e.json", "w") as outfile:
        outfile.write(json_object)

    torch.save({
            'causal_model': att_net.state_dict(),
            'predictco': g.state_dict(),
            'model_optimizer':model_optimizer.state_dict(),
            'conf_opt':conf_opt.state_dict()
            }, '/content/drive/MyDrive/running_RQ2/running_Graph_sst2/running_DIR/Gsst2allmodel_DIR.pt')
    if(epoch>50):
      check=abs(acc-loss_value_valation[len(loss_value_valation)-50])/50
      if(check<=0.0001):
        break

g.eval()
att_net.eval()
acc = 0
for graph in val_loader:
  graph.to(device)
  (causal_x, causal_edge_index, causal_edge_attr, causal_edge_weight, causal_batch),\
  (conf_x, conf_edge_index, conf_edge_attr, conf_edge_weight, conf_batch), edge_score = att_net(graph)
  set_masks(causal_edge_weight, g)
  out = g(x=causal_x, edge_index=causal_edge_index,
          edge_attr=causal_edge_attr, batch=causal_batch)
  clear_masks(g)
  acc += torch.sum(out.argmax(-1).view(-1) == graph.y.view(-1))
acc = float(acc) / len(val_loader.dataset)
loss_value_valation.append(acc)
print(f"causal test accuracy:{acc}")
dictionary={"number of epoch":epoch,
            "training loss list":loss_value,
            "valation accuracy list":loss_value_valation,
            "test accuracy value":acc}

# Serializing json
json_object = json.dumps(dictionary,indent=4)

# Writing to sample.json
with open("/content/drive/MyDrive/running_RQ2/running_Graph_sst2/running_DIR/Gsst2DIR_tl_va_e.json", "w") as outfile:
    outfile.write(json_object)








''' conf_opt.zero_grad()
all_conf_loss.backward()
conf_opt.step()

model_optimizer.zero_grad()
all_loss.backward()
model_optimizer.step()'''





-----training-------0


100%|██████████| 222/222 [00:39<00:00,  5.56it/s]


number of 0 of all loss:0.6924734115600586
causal val accuracy:0.6117025599349858
-----training-------1


100%|██████████| 222/222 [00:42<00:00,  5.25it/s]


number of 1 of all loss:0.6890703439712524
causal val accuracy:0.6164973587972369
-----training-------2


...

100%|██████████| 222/222 [00:43<00:00,  5.12it/s]


number of 72 of all loss:0.5082230567932129
causal val accuracy:0.8115400243803332
-----training-------73


100%|██████████| 222/222 [00:44<00:00,  4.99it/s]


number of 73 of all loss:0.5161728858947754
causal val accuracy:0.8104835432750914
causal test accuracy:0.8104835432750914


' conf_opt.zero_grad()\nall_conf_loss.backward()\nconf_opt.step()\n\nmodel_optimizer.zero_grad()\nall_loss.backward()\nmodel_optimizer.step()'

In [ ]:
'''test_loader = DataLoader(testing_final, batch_size=1, shuffle=False)

g.eval()
att_net.eval()
acc = 0
for graph in test_loader:
  graph.to(device)
  (causal_x, causal_edge_index, causal_edge_attr, causal_edge_weight, causal_batch),\
  (conf_x, conf_edge_index, conf_edge_attr, conf_edge_weight, conf_batch), edge_score = att_net(graph)
  set_masks(causal_edge_weight, g)
  out = g(x=causal_x, edge_index=causal_edge_index,
          edge_attr=causal_edge_attr, batch=causal_batch)
  clear_masks(g)
  acc += torch.sum(out.argmax(-1).view(-1) == graph.y.view(-1))
acc = float(acc) / len(test_loader.dataset)
loss_value_valation.append(acc)
print(f"causal test accuracy:{acc}")
dictionary={"number of epoch":epoch,
            "training loss list":loss_value,
            "valation accuracy list":loss_value_valation,
            "test accuracy value":acc}

# Serializing json
json_object = json.dumps(dictionary,indent=4)

# Writing to sample.json
with open("/content/drive/MyDrive/running_dir_mnist/numberDIR_tl_va_e08.json", "w") as outfile:
    outfile.write(json_object)
'''






'test_loader = DataLoader(testing_final, batch_size=1, shuffle=False)\n\ng.eval()\natt_net.eval()\nacc = 0\nfor graph in test_loader: \n  graph.to(device)\n  (causal_x, causal_edge_index, causal_edge_attr, causal_edge_weight, causal_batch),  (conf_x, conf_edge_index, conf_edge_attr, conf_edge_weight, conf_batch), edge_score = att_net(graph)\n  set_masks(causal_edge_weight, g)\n  out = g(x=causal_x, edge_index=causal_edge_index, \n          edge_attr=causal_edge_attr, batch=causal_batch)\n  clear_masks(g)\n  acc += torch.sum(out.argmax(-1).view(-1) == graph.y.view(-1))\nacc = float(acc) / len(test_loader.dataset)\nloss_value_valation.append(acc)\nprint(f"causal test accuracy:{acc}")\ndictionary={"number of epoch":epoch,\n            "training loss list":loss_value,\n            "valation accuracy list":loss_value_valation,\n            "test accuracy value":acc}\n \n# Serializing json\njson_object = json.dumps(dictionary,indent=4)\n\n# Writing to sample.json\nwith open("/content/drive/M

In [ ]:
acc

0.8251930109711499

In [ ]:
dd

NameError: ignored

In [ ]:
torch.save(att_net.state_dict(),'/content/drive/MyDrive/Colab_Notebooks/dir0123causl.pt')
torch.save(g.state_dict(),'/content/drive/MyDrive/Colab_Notebooks/dir0123pred.pt')

In [ ]:
new=[]
for i in range(len(l)):
  new.append(float(l[i]))
line=np.array(new)
plt.plot(line)
plt.show()

In [ ]:
t2=DataLoader(training_final[20000:], batch_size=1, shuffle=False)
t2_load=[]
for i in t2:
  t2_load.append(i)

In [ ]:
len(t2_load)

In [ ]:
causal_acc=0
conf_acc=0
n_test_data=0
for graph in t2_load:
  graph.to(device)
  n_test_data+=1


  (causal_x, causal_edge_index, causal_edge_attr, causal_edge_weight, causal_batch),\
  (conf_x, conf_edge_index, conf_edge_attr, conf_edge_weight, conf_batch), edge_score = att_net(graph)

  set_masks(causal_edge_weight, g)
  causal_out = g(
      x=causal_x, edge_index=causal_edge_index,
      edge_attr=causal_edge_attr, batch=causal_batch)
  set_masks(conf_edge_weight, g)
  conf_out = g(x=conf_x, edge_index=conf_edge_index,
          edge_attr=conf_edge_attr, batch=conf_batch)
  clear_masks(g)
  causal_acc += torch.sum(causal_out.argmax(-1).view(-1) == graph.y.view(-1))
  conf_acc += torch.sum(conf_out.argmax(-1).view(-1) == graph.y.view(-1))



In [ ]:
causal_acc/4754

In [ ]:
#newdata_with_feat_knn=[]
for i in range(len(data_with_feat_knn.labels)):
  if(data_with_feat_knn[i][1]==0):
    sp_data=data_with_feat_knn.sp_data[i]
    adj_matrix=data_with_feat_knn.Adj_matrices[i]
    label=data_with_feat_knn[i][1]
    feat_coord=data_with_feat_knn.use_feat_for_graph
    break


In [ ]:
sp_data[1].shape

In [ ]:

t1=DataLoader(new_012, batch_size=1, shuffle=False)
t_load=[]
for i in t1:
  t_load.append(i)

In [ ]:
idex=0
(causal_x, causal_edge_index, causal_edge_attr, causal_edge_weight, causal_batch),\
      (conf_x, conf_edge_index, conf_edge_attr, conf_edge_weight, conf_batch), edge_score = att_net(t_load[idex].to(device))

In [ ]:
t_load[0]

In [ ]:
causal_node=np.array(torch.unique(causal_edge_index).cpu())
print(causal_node)

In [ ]:
causal_node=np.array(torch.unique(causal_edge_index).cpu())
print(causal_node)

In [ ]:
ncausal_node=np.array(torch.unique(conf_edge_index).cpu())
print(ncausal_node)

In [ ]:
conf_x.detach().numpy().sum(axis=1)

In [ ]:
plt.imshow(training_data_01[idex])
print("Label: ",training_label_01[idex])

In [ ]:
sample = np.random.choice(len(data_with_feat_knn))
g_sample = data_with_feat_knn[idex][0]
print("Label: ", data_with_feat_knn[idex][1])
nx.draw(g_sample.to_networkx(), with_labels=True)
plt.show()

In [ ]:
import networkx as nx
from scipy.spatial.distance import pdist, squareform
from pylab import rcParams
def plot_superpixels_graph(sp_data, adj_matrix, label, feat_coord, with_edges,real,conf_x):
    Y = squareform(pdist(sp_data[1], 'euclidean'))
    x_coord = sp_data[1] #np.flip(dataset.train.sp_data[_][1], 1)
    #intensities = sp_data[0].mean(axis=1)
    #label = classes[label]
    if real:
      intensities = sp_data[0].mean(axis=1)
    else:
      intensities=np.array([0]*sp_data[0].shape[0])
      intensities[ncausal_node]=conf_x.detach().numpy()
      #intensities=np.array(conf_x.sum(dim=1).cpu().detach())




    G = nx.from_numpy_array(Y)
    pos = dict(zip(range(len(x_coord)), x_coord.tolist()))
    rotated_pos = {node: (y,-x) for (node, (x,y)) in pos.items()} # rotate the coords by 90 degree

    edge_list = []
    for src, dsts in enumerate(compute_edges_list(adj_matrix)[0]):
        for dst in dsts:
            edge_list.append((src, dst))

    nx.draw_networkx_nodes(G, rotated_pos, node_color=intensities, cmap=matplotlib.cm.Reds, node_size=60) #len(intensities))
    if with_edges:
        nx.draw_networkx_edges(G, rotated_pos, edge_list, alpha=0.3)




In [ ]:

#sp_data=data_with_feat_knn.sp_data[idex]
#adj_matrix=data_with_feat_knn.Adj_matrices[idex]
#label=data_with_feat_knn[idex][1]
#feat_coord=data_with_feat_knn.use_feat_for_graph
Y = squareform(pdist(sp_data[1], 'euclidean'))
x_coord = sp_data[1] #np.flip(dataset.train.sp_data[_][1], 1)
#intensities = sp_data[0].mean(axis=1)
#label = classes[label]


intensities=np.array([0]*sp_data[0].shape[0])
intensities[causal_node]=causal_x.detach().numpy().sum(axis=1)
  #intensities=np.array(conf_x.sum(dim=1).cpu().detach())




G = nx.from_numpy_array(Y)
pos = dict(zip(range(len(x_coord)), x_coord.tolist()))
rotated_pos = {node: (y,-x) for (node, (x,y)) in pos.items()} # rotate the coords by 90 degree

edge_list = []
for src, dsts in enumerate(compute_edges_list(adj_matrix)[0]):
    for dst in dsts:
        edge_list.append((src, dst))

nx.draw_networkx_nodes(G, rotated_pos, node_color=intensities, cmap=matplotlib.cm.Reds, node_size=60) #len(intensities))
if True:
    nx.draw_networkx_edges(G, rotated_pos, edge_list, alpha=0.3)
#with_edges=True
#plt.figure(figsize=(5, 5))
#real=False
#plot_superpixels_graph(sp_data, adj_matrix, label, feat_coord, with_edges,real,conf_x)

In [ ]:
plt.figure(figsize=(5, 5))
real=True
plot_superpixels_graph(sp_data, adj_matrix, label, feat_coord, with_edges,real,causal_node)